In [1]:
from TestProblems import prey_predator_model, van_der_pol_model, CSTR_1state_model, CSTR_3state_model
from Solvers import ESDIRK
from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt

# ESDIRK23